### Saving a Custom Software Specification to a Project

This notebook creates a custom software specification and makes it available to the Project. This is used for saving models or functions that require libraries not already preinstalled in existing software specifications.

In this example, we are create a software specification that includes the machine learning library `pyod`.

We use the Watson Machine Learning API to create our software specification. The below code imports this library and creates a client using the required credential information.

In [1]:
import os
import ibm_watson_machine_learning

token = os.environ['USER_ACCESS_TOKEN']

wml_credentials = {
    "token": token,
    "instance_id" : "openshift",
    "url": os.environ['RUNTIME_ENV_APSX_URL'],
    "version": "3.5"
}

wml_client = ibm_watson_machine_learning.APIClient(wml_credentials)
wml_client.version

'1.0.34'

We capture the project ID for where this notebook is currently running.

In [2]:
project_id = os.environ['PROJECT_ID']

We tell the client we wish to use this project as the default so that we can add a new software specification here. We set some default `undefined` values for our new software specification (`asset_id`) and package extension ID (`pe_asset_id`)

In [3]:
wml_client.set.default_project(project_id)

asset_id = 'undefined'
pe_asset_id = 'undefined'

We can view the current list of software specifications.

In [4]:
wml_client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
tensorflow_1.15-py3.6          2b73a275-7cbf-420b-a912-eae7f436e0bc  base
pytorch_1.2-py3.6              2c8ef57d-2687-4b7d-acce-01f94976dac1  base
spark-mllib_2.3                2e51f70

We create the metadata for our package extension, this specifies the name of the extension and how we will provide the appropriate information, using a conda YAML file (`conta_yml`).

In [5]:
# create the metadata for package extensions
pe_metadata = {
 wml_client.package_extensions.ConfigurationMetaNames.NAME: 'pyod custom library',
 wml_client.package_extensions.ConfigurationMetaNames.TYPE: 'conda_yml'
}

Create a YAML file and upload it to the project.

We have created and uploaded `customlibrarypyod.yaml` which contains the following text:

```
name: pyod_env
channels:
  - empty
  - nodefaults
dependencies:
- pip:
  - pyod
```

Alternative or additional dependencies should be provided under `dependencies:` following YAML standards.

We create a new package extension and store it using the WML client. We pass the metadata from above and the path to the YAML file.

We then capture the ID for our package extension, as well as the base software specification we wish to extend (`default_py3.7`) which already includes a foundation of libraries, just not the additional one we need (in this case `pyod`).

In [6]:
pe_asset_details = wml_client.package_extensions.store(meta_props=pe_metadata, file_path='/project_data/data_asset/customlibrarypyod.yaml')
pe_asset_id = wml_client.package_extensions.get_id(pe_asset_details)

base_id = wml_client.software_specifications.get_id_by_name('default_py3.7')

Creating package extensions
SUCCESS


Now that we have create the package extension, we create and store the new software specification. We first create metadata including the name, an optional description and the ID for the base software specification (collected above).

This then gets stores as a new software specification and we capture its ID.

In [7]:
# create the metadata for software specs
ss_metadata = {
 wml_client.software_specifications.ConfigurationMetaNames.NAME: 'Python 3.7 with pyod',
 wml_client.software_specifications.ConfigurationMetaNames.DESCRIPTION: 'Adding pyod as custom library',
 wml_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {'guid': base_id}
}
# store the software spec
ss_asset_details = wml_client.software_specifications.store(meta_props=ss_metadata)
# get the id of the new asset
asset_id = wml_client.software_specifications.get_id(ss_asset_details)

We attach the package extension to our software specification.

In [8]:
wml_client.software_specifications.add_package_extension(asset_id, pe_asset_id)

SUCCESS


'SUCCESS'

We can review the details for our new package.

In [9]:
ss_asset_details = wml_client.software_specifications.get_details(asset_id)
print('Package extensions', ss_asset_details['entity']['software_specification']['package_extensions'])

Package extensions [{'metadata': {'rov': {'mode': 0, 'collaborator_ids': {}}, 'project_id': '7bd837d2-f736-4a67-9a8a-8a95bbb37d9e', 'sandbox_id': '7bd837d2-f736-4a67-9a8a-8a95bbb37d9e', 'usage': {'last_updated_at': '2021-04-12T18:06:19Z', 'last_updater_id': '1000331024', 'last_update_time': 1618250779885, 'last_accessed_at': '2021-04-12T18:06:19Z', 'last_access_time': 1618250779885, 'last_accessor_id': '1000331024', 'access_count': 0}, 'name': 'pyod custom library', 'description': '', 'tags': ['dsx'], 'asset_type': 'package_extension', 'origin_country': 'us', 'rating': 0, 'total_ratings': 0, 'catalog_id': '83b01966-f1c6-4123-a69b-7fc797a5235e', 'created': 0, 'created_at': '2021-04-12T18:06:19Z', 'owner_id': '1000331024', 'size': 79, 'version': 0, 'asset_state': 'available', 'asset_attributes': ['package_extension'], 'asset_id': '9fdb7b9a-10f6-4059-8b19-51ba7360b473', 'asset_category': 'USER'}, 'entity': {'package_extension': {'type': 'conda_yml', 'object_key': 'package_extension/pyod_c

We can view all software specifications to see our custom software specification.

In [10]:
wml_client.software_specifications.list()

-----------------------------  ------------------------------------  -------
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
tensorflow_1.15-py3.6          2b73a275-7cbf-420b-a912-eae7f436e0bc  base
pytorch_1.2-py3.6              2c8ef57d-2687-4b7d-acce-01f94976dac1  base
spark-mllib_2.3                2e51